In [1]:
import numpy as np
import pandas as pd
import re
from unidecode import unidecode

Input data source: [Danh sách đơn vị hành chính cấp huyện của Việt Nam - Wikipedia.org](https://vi.wikipedia.org/wiki/Danh_s%C3%A1ch_%C4%91%C6%A1n_v%E1%BB%8B_h%C3%A0nh_ch%C3%ADnh_c%E1%BA%A5p_huy%E1%BB%87n_c%E1%BB%A7a_Vi%E1%BB%87t_Nam)

Expected output: province, district, ward, long_province, long_district, long_ward, short_district, short_ward, province_english, district_english, ward_english, long_province_english, long_district_english, long_ward_english, short_district_english, short_ward_english, district_level, ward_level, district_level_english, ward_level_english

In [2]:
expected_cols = ['province', 'district', 'ward', 'long_province', 'long_district', 'long_ward', 'short_district', 'short_ward', 'province_english', 'district_english', 'ward_english', 'long_province_english', 'long_district_english', 'long_ward_english', 'short_district_english', 'short_ward_english', 'district_level', 'ward_level', 'district_level_english', 'ward_level_english']

In [3]:
df = pd.read_csv('../data/input/wikipedia.csv')

In [4]:
df

,short_district,province,district_level
0,A Lưới,Thừa Thiên Huế,NaN
1,An Biên,Kiên Giang,NaN
2,An Dương,Hải Phòng,NaN
3,An Khê,Gia Lai,thị xã
4,An Lão,Hải Phòng,NaN
...,...,...,...
699,Yên Phong,Bắc Ninh,NaN
700,Yên Sơn,Tuyên Quang,NaN
701,Yên Thành,Nghệ An,NaN
702,Yên Thế,Bắc Giang,NaN


In [5]:
def create_province_english(text):
    u_text = unidecode(text)
    c_text = str(u_text).title().strip()
    c_text = re.sub(r'\sProvince$|^Tinh\s|^Thanh\sPho\s|\sCity$', '', c_text, flags=re.IGNORECASE)
    # if c_text == 'Ho Chi Minh':
    #     c_text = 'Ho Chi Minh City'
    return c_text

In [6]:
def create_short_district_english(text):
    u_text = unidecode(text)
    c_text = str(u_text).title().strip()
    
    if re.search(r'^Quan \d{1,2}', c_text, flags=re.IGNORECASE):
        c_text = re.sub(r'^Quan ', 'District ', c_text, flags=re.IGNORECASE)
    
    return c_text

In [7]:
df['province'] = df['province'].str.replace('–', '-').str.replace('Thành phố ', '')
df['short_district'] = df['short_district'].str.replace('–', '-')

In [8]:
df['district_level'] = df['district_level'].str.capitalize().str.replace('Huyện đảo', 'Huyện').fillna('Huyện')

In [9]:
df['district_level'].value_counts(dropna=False)

district_level
Huyện        522
Thành phố     85
Thị xã        51
Quận          46
Name: count, dtype: int64

In [10]:
district_count = df[['province', 'short_district']].value_counts().reset_index()
duplicated_districts = district_count[district_count['count'] > 1]['short_district'].tolist()

In [11]:
df_duplicated_districts = df[df.short_district.isin(duplicated_districts)]

In [12]:
df_duplicated_districts

,short_district,province,district_level
74,Cai Lậy,Tiền Giang,Thị xã
75,Cai Lậy,Tiền Giang,Huyện
94,Cao Lãnh,Đồng Tháp,Thành phố
95,Cao Lãnh,Đồng Tháp,Huyện
205,Duyên Hải,Trà Vinh,Thị xã
206,Duyên Hải,Trà Vinh,Huyện
272,Hồng Ngự,Đồng Tháp,Thành phố
273,Hồng Ngự,Đồng Tháp,Huyện
315,Kỳ Anh,Hà Tĩnh,Thị xã
316,Kỳ Anh,Hà Tĩnh,Huyện


In [13]:
# keep_district_levels = ['Thị xã', 'Thành phố']

In [14]:
district_level_english = {
    'Thị xã': 'Town',
    'Thành phố': 'City',
    'Quận': 'District',
    'Huyện': 'District'
}

In [15]:
# df['district_level'] = np.where(df['district_level'].isin(keep_district_levels), df['district_level'], np.nan)
df['district_level_english'] = df['district_level'].map(district_level_english)

In [16]:
df['short_district_english'] = df['short_district'].apply(create_short_district_english)

In [17]:
df['district'] = np.where((df.province.isin(df_duplicated_districts.province)) & (df.short_district.isin(df_duplicated_districts.short_district)), df['district_level'] + ' ' + df['short_district'], df['short_district'])

df['district_english'] = np.where((df.province.isin(df_duplicated_districts.province)) & (df.short_district.isin(df_duplicated_districts.short_district)), df['short_district_english'] + ' ' + df['district_level_english'], df['short_district_english'])

In [18]:
df['province_english'] = df['province'].apply(create_province_english)

In [19]:
df['long_district'] = np.where(~df.short_district.str.contains('|'.join(district_level_english)), df['district_level'] + ' ' + df['short_district'], df['short_district'])

In [20]:
df['long_district_english'] =  np.where(~df.short_district.str.contains('|'.join(district_level_english)), df['short_district_english'] + ' ' + df['district_level_english'], df['short_district_english'])

In [21]:
df = df[[col for col in expected_cols if col in df.columns]]

In [22]:
df

,province,district,long_district,short_district,province_english,district_english,long_district_english,short_district_english,district_level,district_level_english
0,Thừa Thiên Huế,A Lưới,Huyện A Lưới,A Lưới,Thua Thien Hue,A Luoi,A Luoi District,A Luoi,Huyện,District
1,Kiên Giang,An Biên,Huyện An Biên,An Biên,Kien Giang,An Bien,An Bien District,An Bien,Huyện,District
2,Hải Phòng,An Dương,Huyện An Dương,An Dương,Hai Phong,An Duong,An Duong District,An Duong,Huyện,District
3,Gia Lai,An Khê,Thị xã An Khê,An Khê,Gia Lai,An Khe,An Khe Town,An Khe,Thị xã,Town
4,Hải Phòng,An Lão,Huyện An Lão,An Lão,Hai Phong,An Lao,An Lao District,An Lao,Huyện,District
...,...,...,...,...,...,...,...,...,...,...
699,Bắc Ninh,Yên Phong,Huyện Yên Phong,Yên Phong,Bac Ninh,Yen Phong,Yen Phong District,Yen Phong,Huyện,District
700,Tuyên Quang,Yên Sơn,Huyện Yên Sơn,Yên Sơn,Tuyen Quang,Yen Son,Yen Son District,Yen Son,Huyện,District
701,Nghệ An,Yên Thành,Huyện Yên Thành,Yên Thành,Nghe An,Yen Thanh,Yen Thanh District,Yen Thanh,Huyện,District
702,Bắc Giang,Yên Thế,Huyện Yên Thế,Yên Thế,Bac Giang,Yen The,Yen The District,Yen The,Huyện,District


In [23]:
# df.to_csv('../data/output/district_administrative_unit_of_vietnam.csv', index=False, encoding='utf-8')

# Ward

In [24]:
df_ward = pd.read_csv('../data/input/danhmuchanhchinhgso.gov.vn.csv')

In [25]:
df_ward

,long_province,long_district,long_ward,ward_level
0,Thành phố Hà Nội,Quận Ba Đình,Phường Phúc Xá,Phường
1,Thành phố Hà Nội,Quận Ba Đình,Phường Trúc Bạch,Phường
2,Thành phố Hà Nội,Quận Ba Đình,Phường Vĩnh Phúc,Phường
3,Thành phố Hà Nội,Quận Ba Đình,Phường Cống Vị,Phường
4,Thành phố Hà Nội,Quận Ba Đình,Phường Liễu Giai,Phường
...,...,...,...,...
10542,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An Đông,Xã
10543,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An,Xã
10544,Tỉnh Cà Mau,Huyện Ngọc Hiển,Thị trấn Rạch Gốc,Thị trấn
10545,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Tân Ân,Xã


In [26]:
df_ward.ward_level.value_counts(dropna=False)

ward_level
Xã          8149
Phường      1775
Thị trấn     618
Huyện          5
Name: count, dtype: int64

In [27]:
df_ward[df_ward.ward_level == 'Huyện']
# These are islands with no wards

,long_province,long_district,long_ward,ward_level
3795,Thành phố Hải Phòng,Huyện Bạch Long Vĩ,NaN,Huyện
6154,Tỉnh Quảng Trị,Huyện Cồn Cỏ,NaN,Huyện
6352,Thành phố Đà Nẵng,Huyện Hoàng Sa,NaN,Huyện
6767,Tỉnh Quảng Ngãi,Huyện Lý Sơn,NaN,Huyện
8632,Tỉnh Bà Rịa - Vũng Tàu,Huyện Côn Đảo,NaN,Huyện


In [28]:
def create_short_ward(text):
    if not isinstance(text, str):
        return text
    
    if not re.search(r'^Phường \d{1,2}', text, flags=re.IGNORECASE):
        c_text = re.sub(r'^Phường |^Thị trấn |^Xã |^Huyện', '', text, flags=re.IGNORECASE)
    else:
        c_text = text
        
    return c_text

In [29]:
def create_short_ward_english(text):
    if not isinstance(text, str):
        return text
    
    c_text = create_short_ward(text)
    c_text = unidecode(c_text)
    
    if re.search(r'^Phuong \d{1,2}', c_text, flags=re.IGNORECASE):
        c_text = c_text.replace('Phuong', 'Ward')
        
    return c_text

In [30]:
create_short_ward_english('Phường 5')

'Ward 5'

In [31]:
replacements = [
    # Một số từ thường thấy và merge bị missing với data của Wiki
    ('Thuỷ', 'Thủy'),
    ('Thuỵ', 'Thụy'),
    ('Đa Krông', 'Đakrông'),
    ('Đắk Glei', 'Đăk Glei'),
    ('Đắk Tô', 'Đăk Tô'),
    ('Đắk Hà', 'Đăk Hà'),
    ('KBang', 'Kbang'),
    ('Quí', 'Quý'),
    ('Hoà', 'Hòa'),
    ('Đăk Đoa', 'Đak Đoa'),
    ('Đăk Pơ', 'Đak Pơ'),
    ("M'Đrắk", "M'Drắk"),
    ('Krông A Na', 'Krông Ana'),
    ('Đăk Glong', 'Đắk Glong'),
    ("Đắk R'Lấp", "Đắk R'lấp"),
    ('Căng Chải', 'Cang Chải'),
    
    # Một số từ dùng API kiểm tra chính tả của CocCoc

    ("H'đinh", "H'đing"),
    ('Hoả', 'Hỏa'),
    ('Khoá', 'Khóa'),
    ('Hoá', 'Hóa'),
    ('Nà Ơt', 'Nà Ớt')
]

In [32]:
def fix_grammar(text):
    if not isinstance(text, str):
        return text
    c_text = text.strip()
    
    for replacement in replacements:
        c_text = re.sub(rf'\b{replacement[0]}\b', replacement[1], c_text)

    c_text = re.sub(r'^Thị Xã', 'Thị xã', c_text)
    c_text = re.sub(r'^Thành Phố', 'Thành phố', c_text)
    c_text = re.sub(r'^Thị Trấn', 'Thị trấn', c_text)
    c_text = c_text.replace('-', ' - ').replace('  ', ' ').replace(' 0', ' ').replace("' ", "'")
    return c_text

In [33]:
fix_grammar('Thị Xã Quí')

'Thị xã Quý'

In [34]:
def create_province(text):
    c_text = str(text).title().strip()
    c_text = re.sub(r'\sProvince$|^Tỉnh\s|^Thành\sPhố\s|\sCity$', '', c_text, flags=re.IGNORECASE)

    return c_text

In [35]:
df_ward['province'] = df_ward['long_province'].apply(create_province)

In [36]:
for col in df_ward.columns.tolist():
    df_ward[col] = df_ward[col].apply(fix_grammar)

In [37]:
df_ward['short_ward'] = df_ward['long_ward'].apply(create_short_ward)
df_ward['short_ward_english'] = df_ward['long_ward'].apply(create_short_ward_english)

In [38]:
df_ward

,long_province,long_district,long_ward,ward_level,province,short_ward,short_ward_english
0,Thành phố Hà Nội,Quận Ba Đình,Phường Phúc Xá,Phường,Hà Nội,Phúc Xá,Phuc Xa
1,Thành phố Hà Nội,Quận Ba Đình,Phường Trúc Bạch,Phường,Hà Nội,Trúc Bạch,Truc Bach
2,Thành phố Hà Nội,Quận Ba Đình,Phường Vĩnh Phúc,Phường,Hà Nội,Vĩnh Phúc,Vinh Phuc
3,Thành phố Hà Nội,Quận Ba Đình,Phường Cống Vị,Phường,Hà Nội,Cống Vị,Cong Vi
4,Thành phố Hà Nội,Quận Ba Đình,Phường Liễu Giai,Phường,Hà Nội,Liễu Giai,Lieu Giai
...,...,...,...,...,...,...,...
10542,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An Đông,Xã,Cà Mau,Viên An Đông,Vien An Dong
10543,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An,Xã,Cà Mau,Viên An,Vien An
10544,Tỉnh Cà Mau,Huyện Ngọc Hiển,Thị trấn Rạch Gốc,Thị trấn,Cà Mau,Rạch Gốc,Rach Goc
10545,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Tân Ân,Xã,Cà Mau,Tân Ân,Tan An


In [39]:
ward_count = df_ward[['long_province', 'long_district', 'short_ward']].value_counts().reset_index()

In [40]:
df_duplicated_wards = ward_count[ward_count['count'] > 1]

In [41]:
df_duplicated_wards

,long_province,long_district,short_ward,count
0,Tỉnh Cà Mau,Thành phố Cà Mau,Tân Thành,2
1,Thành phố Hà Nội,Huyện Gia Lâm,Yên Viên,2
2,Tỉnh Kiên Giang,Huyện Vĩnh Thuận,Vĩnh Thuận,2
3,Tỉnh Lâm Đồng,Huyện Đạ Huoai,Ma Đa Guôi,2
4,Tỉnh Sóc Trăng,Huyện Long Phú,Long Phú,2
5,Tỉnh Yên Bái,Huyện Yên Bình,Yên Bình,2
6,Tỉnh Đồng Tháp,Huyện Cao Lãnh,Mỹ Thọ,2
7,Tỉnh Đồng Tháp,Huyện Tháp Mười,Mỹ An,2
8,Tỉnh Sơn La,Huyện Mai Sơn,Hát Lót,2
9,Tỉnh Lai Châu,Huyện Mường Tè,Mường Tè,2


In [42]:
df_ward[(df_ward.long_province.isin(df_duplicated_wards.long_province)) & (df_ward.long_district.isin(df_duplicated_wards.long_district)) & (df_ward.short_ward.isin(df_duplicated_wards.short_ward))]

,long_province,long_district,long_ward,ward_level,province,short_ward,short_ward_english
176,Thành phố Hà Nội,Huyện Gia Lâm,Thị trấn Yên Viên,Thị trấn,Hà Nội,Yên Viên,Yen Vien
178,Thành phố Hà Nội,Huyện Gia Lâm,Xã Yên Viên,Xã,Hà Nội,Yên Viên,Yen Vien
1479,Tỉnh Lai Châu,Huyện Mường Tè,Thị trấn Mường Tè,Thị trấn,Lai Châu,Mường Tè,Muong Te
1484,Tỉnh Lai Châu,Huyện Mường Tè,Xã Mường Tè,Xã,Lai Châu,Mường Tè,Muong Te
1706,Tỉnh Sơn La,Huyện Mai Sơn,Thị trấn Hát Lót,Thị trấn,Sơn La,Hát Lót,Hat Lot
1716,Tỉnh Sơn La,Huyện Mai Sơn,Xã Hát Lót,Xã,Sơn La,Hát Lót,Hat Lot
1882,Tỉnh Yên Bái,Huyện Trạm Tấu,Thị trấn Trạm Tấu,Thị trấn,Yên Bái,Trạm Tấu,Tram Tau
1887,Tỉnh Yên Bái,Huyện Trạm Tấu,Xã Trạm Tấu,Xã,Yên Bái,Trạm Tấu,Tram Tau
1918,Tỉnh Yên Bái,Huyện Yên Bình,Thị trấn Yên Bình,Thị trấn,Yên Bái,Yên Bình,Yen Binh
1937,Tỉnh Yên Bái,Huyện Yên Bình,Xã Yên Bình,Xã,Yên Bái,Yên Bình,Yen Binh


In [43]:
ward_level_english = {
    'Xã': 'Commune',
    'Thị trấn': 'Town',
    'Phường': 'Ward',
    'Huyện': 'District'
}

In [44]:
df_ward['ward_level_english'] = df_ward['ward_level'].map(ward_level_english)

In [45]:
df_ward

,long_province,long_district,long_ward,ward_level,province,short_ward,short_ward_english,ward_level_english
0,Thành phố Hà Nội,Quận Ba Đình,Phường Phúc Xá,Phường,Hà Nội,Phúc Xá,Phuc Xa,Ward
1,Thành phố Hà Nội,Quận Ba Đình,Phường Trúc Bạch,Phường,Hà Nội,Trúc Bạch,Truc Bach,Ward
2,Thành phố Hà Nội,Quận Ba Đình,Phường Vĩnh Phúc,Phường,Hà Nội,Vĩnh Phúc,Vinh Phuc,Ward
3,Thành phố Hà Nội,Quận Ba Đình,Phường Cống Vị,Phường,Hà Nội,Cống Vị,Cong Vi,Ward
4,Thành phố Hà Nội,Quận Ba Đình,Phường Liễu Giai,Phường,Hà Nội,Liễu Giai,Lieu Giai,Ward
...,...,...,...,...,...,...,...,...
10542,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An Đông,Xã,Cà Mau,Viên An Đông,Vien An Dong,Commune
10543,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An,Xã,Cà Mau,Viên An,Vien An,Commune
10544,Tỉnh Cà Mau,Huyện Ngọc Hiển,Thị trấn Rạch Gốc,Thị trấn,Cà Mau,Rạch Gốc,Rach Goc,Town
10545,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Tân Ân,Xã,Cà Mau,Tân Ân,Tan An,Commune


In [46]:
df_ward['ward'] = np.where((df_ward.long_province.isin(df_duplicated_wards.long_province)) & (df_ward.long_district.isin(df_duplicated_wards.long_district)) & (df_ward.short_ward.isin(df_duplicated_wards.short_ward)), df_ward['ward_level'] + ' ' + df_ward['short_ward'], df_ward['short_ward'])

In [47]:
df_ward['ward_english'] = np.where((df_ward.long_province.isin(df_duplicated_wards.long_province)) & (df_ward.long_district.isin(df_duplicated_wards.long_district)) & (df_ward.short_ward.isin(df_duplicated_wards.short_ward)), df_ward['short_ward_english'] + ' ' + df_ward['ward_level_english'], df_ward['short_ward_english'])

In [48]:
df_ward['long_ward_english'] = np.where(
    (df_ward['short_ward'].notna()) & 
    (~df_ward['short_ward'].astype(str).str.contains('|'.join(ward_level_english))),
    df_ward['short_ward_english'].fillna('') + ' ' + df_ward['ward_level_english'],
    df_ward['short_ward_english']
)

In [49]:
def create_long_province_english(text):
    u_text = unidecode(text)
    c_text = str(u_text).title().strip()
    if re.search(r'^Thanh Pho\s', c_text):
        c_text = re.sub(r'^Thanh Pho\s', '', c_text) + ' City'
    elif re.search(r'^Tinh\s', c_text):
        c_text = re.sub(r'^Tinh\s', '', c_text) + ' Province'
    return c_text

In [50]:
create_long_province_english('Thành phố Hà Nội')

'Ha Noi City'

In [51]:
df_ward['long_province_english'] = df_ward['long_province'].apply(create_long_province_english)

In [52]:
df_map = pd.merge(df_ward, df, on=['province', 'long_district'], how='left', suffixes=('', '_df'))

In [53]:
df_map

,long_province,long_district,long_ward,ward_level,province,short_ward,short_ward_english,ward_level_english,ward,ward_english,long_ward_english,long_province_english,district,short_district,province_english,district_english,long_district_english,short_district_english,district_level,district_level_english
0,Thành phố Hà Nội,Quận Ba Đình,Phường Phúc Xá,Phường,Hà Nội,Phúc Xá,Phuc Xa,Ward,Phúc Xá,Phuc Xa,Phuc Xa Ward,Ha Noi City,Ba Đình,Ba Đình,Ha Noi,Ba Dinh,Ba Dinh District,Ba Dinh,Quận,District
1,Thành phố Hà Nội,Quận Ba Đình,Phường Trúc Bạch,Phường,Hà Nội,Trúc Bạch,Truc Bach,Ward,Trúc Bạch,Truc Bach,Truc Bach Ward,Ha Noi City,Ba Đình,Ba Đình,Ha Noi,Ba Dinh,Ba Dinh District,Ba Dinh,Quận,District
2,Thành phố Hà Nội,Quận Ba Đình,Phường Vĩnh Phúc,Phường,Hà Nội,Vĩnh Phúc,Vinh Phuc,Ward,Vĩnh Phúc,Vinh Phuc,Vinh Phuc Ward,Ha Noi City,Ba Đình,Ba Đình,Ha Noi,Ba Dinh,Ba Dinh District,Ba Dinh,Quận,District
3,Thành phố Hà Nội,Quận Ba Đình,Phường Cống Vị,Phường,Hà Nội,Cống Vị,Cong Vi,Ward,Cống Vị,Cong Vi,Cong Vi Ward,Ha Noi City,Ba Đình,Ba Đình,Ha Noi,Ba Dinh,Ba Dinh District,Ba Dinh,Quận,District
4,Thành phố Hà Nội,Quận Ba Đình,Phường Liễu Giai,Phường,Hà Nội,Liễu Giai,Lieu Giai,Ward,Liễu Giai,Lieu Giai,Lieu Giai Ward,Ha Noi City,Ba Đình,Ba Đình,Ha Noi,Ba Dinh,Ba Dinh District,Ba Dinh,Quận,District
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10542,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An Đông,Xã,Cà Mau,Viên An Đông,Vien An Dong,Commune,Viên An Đông,Vien An Dong,Vien An Dong Commune,Ca Mau Province,Ngọc Hiển,Ngọc Hiển,Ca Mau,Ngoc Hien,Ngoc Hien District,Ngoc Hien,Huyện,District
10543,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An,Xã,Cà Mau,Viên An,Vien An,Commune,Viên An,Vien An,Vien An Commune,Ca Mau Province,Ngọc Hiển,Ngọc Hiển,Ca Mau,Ngoc Hien,Ngoc Hien District,Ngoc Hien,Huyện,District
10544,Tỉnh Cà Mau,Huyện Ngọc Hiển,Thị trấn Rạch Gốc,Thị trấn,Cà Mau,Rạch Gốc,Rach Goc,Town,Rạch Gốc,Rach Goc,Rach Goc Town,Ca Mau Province,Ngọc Hiển,Ngọc Hiển,Ca Mau,Ngoc Hien,Ngoc Hien District,Ngoc Hien,Huyện,District
10545,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Tân Ân,Xã,Cà Mau,Tân Ân,Tan An,Commune,Tân Ân,Tan An,Tan An Commune,Ca Mau Province,Ngọc Hiển,Ngọc Hiển,Ca Mau,Ngoc Hien,Ngoc Hien District,Ngoc Hien,Huyện,District


In [54]:
df_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10547 entries, 0 to 10546
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   long_province           10547 non-null  object
 1   long_district           10547 non-null  object
 2   long_ward               10542 non-null  object
 3   ward_level              10547 non-null  object
 4   province                10547 non-null  object
 5   short_ward              10542 non-null  object
 6   short_ward_english      10542 non-null  object
 7   ward_level_english      10547 non-null  object
 8   ward                    10542 non-null  object
 9   ward_english            10542 non-null  object
 10  long_ward_english       10542 non-null  object
 11  long_province_english   10547 non-null  object
 12  district                10547 non-null  object
 13  short_district          10547 non-null  object
 14  province_english        10547 non-null  object
 15  di

In [55]:
df_map[df_map.district_english.isna()][['province', 'long_district']].drop_duplicates()

,province,long_district


In [56]:
df[df.province_english.str.contains('Dak Nong') & df.district_english.str.contains('Lap')]

,province,district,long_district,short_district,province_english,district_english,long_district_english,short_district_english,district_level,district_level_english
155,Đắk Nông,Đắk R'lấp,Huyện Đắk R'lấp,Đắk R'lấp,Dak Nong,Dak R'Lap,Dak R'Lap District,Dak R'Lap,Huyện,District


In [57]:
df_map

,long_province,long_district,long_ward,ward_level,province,short_ward,short_ward_english,ward_level_english,ward,ward_english,long_ward_english,long_province_english,district,short_district,province_english,district_english,long_district_english,short_district_english,district_level,district_level_english
0,Thành phố Hà Nội,Quận Ba Đình,Phường Phúc Xá,Phường,Hà Nội,Phúc Xá,Phuc Xa,Ward,Phúc Xá,Phuc Xa,Phuc Xa Ward,Ha Noi City,Ba Đình,Ba Đình,Ha Noi,Ba Dinh,Ba Dinh District,Ba Dinh,Quận,District
1,Thành phố Hà Nội,Quận Ba Đình,Phường Trúc Bạch,Phường,Hà Nội,Trúc Bạch,Truc Bach,Ward,Trúc Bạch,Truc Bach,Truc Bach Ward,Ha Noi City,Ba Đình,Ba Đình,Ha Noi,Ba Dinh,Ba Dinh District,Ba Dinh,Quận,District
2,Thành phố Hà Nội,Quận Ba Đình,Phường Vĩnh Phúc,Phường,Hà Nội,Vĩnh Phúc,Vinh Phuc,Ward,Vĩnh Phúc,Vinh Phuc,Vinh Phuc Ward,Ha Noi City,Ba Đình,Ba Đình,Ha Noi,Ba Dinh,Ba Dinh District,Ba Dinh,Quận,District
3,Thành phố Hà Nội,Quận Ba Đình,Phường Cống Vị,Phường,Hà Nội,Cống Vị,Cong Vi,Ward,Cống Vị,Cong Vi,Cong Vi Ward,Ha Noi City,Ba Đình,Ba Đình,Ha Noi,Ba Dinh,Ba Dinh District,Ba Dinh,Quận,District
4,Thành phố Hà Nội,Quận Ba Đình,Phường Liễu Giai,Phường,Hà Nội,Liễu Giai,Lieu Giai,Ward,Liễu Giai,Lieu Giai,Lieu Giai Ward,Ha Noi City,Ba Đình,Ba Đình,Ha Noi,Ba Dinh,Ba Dinh District,Ba Dinh,Quận,District
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10542,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An Đông,Xã,Cà Mau,Viên An Đông,Vien An Dong,Commune,Viên An Đông,Vien An Dong,Vien An Dong Commune,Ca Mau Province,Ngọc Hiển,Ngọc Hiển,Ca Mau,Ngoc Hien,Ngoc Hien District,Ngoc Hien,Huyện,District
10543,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An,Xã,Cà Mau,Viên An,Vien An,Commune,Viên An,Vien An,Vien An Commune,Ca Mau Province,Ngọc Hiển,Ngọc Hiển,Ca Mau,Ngoc Hien,Ngoc Hien District,Ngoc Hien,Huyện,District
10544,Tỉnh Cà Mau,Huyện Ngọc Hiển,Thị trấn Rạch Gốc,Thị trấn,Cà Mau,Rạch Gốc,Rach Goc,Town,Rạch Gốc,Rach Goc,Rach Goc Town,Ca Mau Province,Ngọc Hiển,Ngọc Hiển,Ca Mau,Ngoc Hien,Ngoc Hien District,Ngoc Hien,Huyện,District
10545,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Tân Ân,Xã,Cà Mau,Tân Ân,Tan An,Commune,Tân Ân,Tan An,Tan An Commune,Ca Mau Province,Ngọc Hiển,Ngọc Hiển,Ca Mau,Ngoc Hien,Ngoc Hien District,Ngoc Hien,Huyện,District


In [58]:
len(expected_cols)

20

In [59]:
[i for i in expected_cols if i not in df_map.columns.tolist()]

[]

In [60]:
df_map = df_map[expected_cols]

In [61]:
df_map

,province,district,ward,long_province,long_district,long_ward,short_district,short_ward,province_english,district_english,ward_english,long_province_english,long_district_english,long_ward_english,short_district_english,short_ward_english,district_level,ward_level,district_level_english,ward_level_english
0,Hà Nội,Ba Đình,Phúc Xá,Thành phố Hà Nội,Quận Ba Đình,Phường Phúc Xá,Ba Đình,Phúc Xá,Ha Noi,Ba Dinh,Phuc Xa,Ha Noi City,Ba Dinh District,Phuc Xa Ward,Ba Dinh,Phuc Xa,Quận,Phường,District,Ward
1,Hà Nội,Ba Đình,Trúc Bạch,Thành phố Hà Nội,Quận Ba Đình,Phường Trúc Bạch,Ba Đình,Trúc Bạch,Ha Noi,Ba Dinh,Truc Bach,Ha Noi City,Ba Dinh District,Truc Bach Ward,Ba Dinh,Truc Bach,Quận,Phường,District,Ward
2,Hà Nội,Ba Đình,Vĩnh Phúc,Thành phố Hà Nội,Quận Ba Đình,Phường Vĩnh Phúc,Ba Đình,Vĩnh Phúc,Ha Noi,Ba Dinh,Vinh Phuc,Ha Noi City,Ba Dinh District,Vinh Phuc Ward,Ba Dinh,Vinh Phuc,Quận,Phường,District,Ward
3,Hà Nội,Ba Đình,Cống Vị,Thành phố Hà Nội,Quận Ba Đình,Phường Cống Vị,Ba Đình,Cống Vị,Ha Noi,Ba Dinh,Cong Vi,Ha Noi City,Ba Dinh District,Cong Vi Ward,Ba Dinh,Cong Vi,Quận,Phường,District,Ward
4,Hà Nội,Ba Đình,Liễu Giai,Thành phố Hà Nội,Quận Ba Đình,Phường Liễu Giai,Ba Đình,Liễu Giai,Ha Noi,Ba Dinh,Lieu Giai,Ha Noi City,Ba Dinh District,Lieu Giai Ward,Ba Dinh,Lieu Giai,Quận,Phường,District,Ward
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10542,Cà Mau,Ngọc Hiển,Viên An Đông,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An Đông,Ngọc Hiển,Viên An Đông,Ca Mau,Ngoc Hien,Vien An Dong,Ca Mau Province,Ngoc Hien District,Vien An Dong Commune,Ngoc Hien,Vien An Dong,Huyện,Xã,District,Commune
10543,Cà Mau,Ngọc Hiển,Viên An,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An,Ngọc Hiển,Viên An,Ca Mau,Ngoc Hien,Vien An,Ca Mau Province,Ngoc Hien District,Vien An Commune,Ngoc Hien,Vien An,Huyện,Xã,District,Commune
10544,Cà Mau,Ngọc Hiển,Rạch Gốc,Tỉnh Cà Mau,Huyện Ngọc Hiển,Thị trấn Rạch Gốc,Ngọc Hiển,Rạch Gốc,Ca Mau,Ngoc Hien,Rach Goc,Ca Mau Province,Ngoc Hien District,Rach Goc Town,Ngoc Hien,Rach Goc,Huyện,Thị trấn,District,Town
10545,Cà Mau,Ngọc Hiển,Tân Ân,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Tân Ân,Ngọc Hiển,Tân Ân,Ca Mau,Ngoc Hien,Tan An,Ca Mau Province,Ngoc Hien District,Tan An Commune,Ngoc Hien,Tan An,Huyện,Xã,District,Commune


In [62]:
df_map.to_csv('../data/output/vietnam_administrative_units.csv', index=False, encoding='utf-8')

In [63]:
df_ward_2 = df_ward[['province', 'long_district']].drop_duplicates()

In [64]:
df_ward_2['check'] = 1

In [65]:
df_map_2 = pd.merge(df, df_ward_2, on=['province', 'long_district'], how='left')

In [66]:
df_map_2[df_map_2.check.isna()]

,province,district,long_district,short_district,province_english,district_english,long_district_english,short_district_english,district_level,district_level_english,check
